This notebook was inspired by this [LlamaIndex example](https://docs.llamaindex.ai/en/latest/examples/cookbooks/mistralai/).

Making some changes to it with the only intention of trying ideas and learning.

Notice that I am assuming you have the relevant API_KEYs as environmental variables.

In [ ]:
%pip install llama-index-embeddings-mistralai

In [14]:
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import Settings
import subprocess
import nest_asyncio
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
import subprocess
import os
nest_asyncio.apply()

## Setup LLM and Embedding Model¶


In [3]:
llm = MistralAI(model="open-mixtral-8x22b", temperature=0.1)
embed_model = MistralAIEmbedding(model_name="mistral-embed")

Settings.llm = llm
Settings.embed_model = embed_model

## Defining global variables

In [5]:
COMPANIES = ["uber", "lyft"]
DATA_DIRS = {}
PERSIST_DIRS = {}
for c in COMPANIES:
    DATA_DIRS[c] = os.path.join(os.environ["DATA_DIR"], f"{c}")
    PERSIST_DIRS[c] = os.path.join(os.environ["PERSIST_DIR"], f"{c}")


## Download data
You can access more files by usinig the next url format

https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/{`COMPANY_NAME`}_10q_{`MONTH`}_{`YEAR`}.pdf'

In [6]:
for c in COMPANIES:
    if not os.path.exists(DATA_DIRS[c]):
        os.mkdir(DATA_DIRS[c])
        command = f"wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/{c}_2021.pdf' -O '{DATA_DIRS[c]}/{c}_2021.pdf'"        
        subprocess.run(command, shell=True)

# Create index and query engine for each company individually

In [9]:
query_engine_dict = {}
for c in COMPANIES:
    if not os.path.exists(PERSIST_DIRS[c]):
        print("Creating Index")
        # load the documents and create the index
        documents = SimpleDirectoryReader(DATA_DIRS[c]).load_data()
        index = VectorStoreIndex.from_documents(documents)
        # store it for later
        index.storage_context.persist(persist_dir=PERSIST_DIRS[c])
    else:
        print("Loading Index")
        # load the existing index
        storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIRS[c])
        index = load_index_from_storage(storage_context)
    
    query_engine_dict[c] = index.as_query_engine(similarity_top_k=5)
    

In [11]:
response = query_engine_dict["uber"].query("What is the revenue of uber in 2021?")
print(response)

Uber's revenue in 2021 was $17,455 million.


In [13]:
response = query_engine_dict["lyft"].query("What are lyft investments in 2021? Show output as a list with main topics.")
print(response)

In 2021, Lyft invested in several areas to advance its mission and maintain its position as a leader in the transportation industry. These investments include:

1. Expansion of Light Vehicles and Lyft Autonomous
      
2. Efficient Operations
      
3. Brand and Social Responsibility
      
4. Electric Vehicles
      
5. Driver Experience
      
6. Marketplace Technology
      
7. Mergers and Acquisitions
      
8. Intellectual Property
      
9. Trademarks and Service Marks
